# Access Link

https://colab.research.google.com/drive/1Et1kxQC3OpryUeQ09RE5oj5-Xh2U8dZm?usp=sharing 

# Install Dependencies

In [1]:
pip install pytube mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Libraries

In [2]:
import cv2 
import math
import numpy as np
import mediapipe as mp
from pytube import YouTube
from google.colab.patches import cv2_imshow

# 2D Angle
  # Angle between point A & C given B
def get_angle(a, b, c):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

# Prepare MediaPipe & Pytube

In [6]:
# Pytube Setup

# Example - Normal Position
yt = YouTube("https://youtu.be/fNxwjJEEeVo")

# Example - Aero Position
#yt = YouTube("https://youtu.be/9_HR4_diEJ4")

# Get best video quality
video_link = yt.streams.filter(adaptive=True)[0]

# Get Name
video_name = yt.title

# MediaPipe Setup 
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Get codes for landmarks
landmark_list = mp_pose.PoseLandmark.__dict__['_member_names_']
landmark_dictionary = dict(zip(landmark_list, range(len(landmark_list))))

# Get Input Video
input_video = cv2.VideoCapture(video_link.url)

# Get Width, Height & FPS
frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(input_video.get(cv2.CAP_PROP_FPS))

# Font type
font = cv2.FONT_HERSHEY_SIMPLEX
red = (50, 50, 255)
green = (127, 255, 0)

# Video Format
fourcc = cv2.VideoWriter_fourcc(*'MP4V')

# Define Output Video
output_video = cv2.VideoWriter(video_name + " " + "processed.mp4",
                               fourcc,
                               fps, 
                               (frame_width, frame_height))

# Run Model & Video

In [7]:
# Define Model 
with mp_pose.Pose(min_detection_confidence=0.8, 
                  min_tracking_confidence=0.8, 
                  enable_segmentation=True, 
                  model_complexity=2) as pose:
  # While true
  while input_video.isOpened():
    check, frame = input_video.read()

    # If end of video then break loop
    if check is False: break

    # Recolor Feed
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Make Detections
    results = pose.process(image)

    # Recolor image back to BGR for rendering
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Pose Detections
    mp_drawing.draw_landmarks(image, 
                              results.pose_landmarks, 
                              mp_pose.POSE_CONNECTIONS, 
                              landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

    # If model detected pose
    if results.pose_landmarks:

      # Right Shoulder
      rs = [results.pose_landmarks.landmark[landmark_dictionary['RIGHT_SHOULDER']].x,
            results.pose_landmarks.landmark[landmark_dictionary['RIGHT_SHOULDER']].y,
            results.pose_landmarks.landmark[landmark_dictionary['RIGHT_SHOULDER']].z]
      # Right Hip
      rh = [results.pose_landmarks.landmark[landmark_dictionary['RIGHT_HIP']].x,
            results.pose_landmarks.landmark[landmark_dictionary['RIGHT_HIP']].y,
            results.pose_landmarks.landmark[landmark_dictionary['RIGHT_HIP']].z]
      # Right Wirst
      rw = [results.pose_landmarks.landmark[landmark_dictionary['RIGHT_WRIST']].x,
            results.pose_landmarks.landmark[landmark_dictionary['RIGHT_WRIST']].y,
            results.pose_landmarks.landmark[landmark_dictionary['RIGHT_WRIST']].z]
      # Left Shoulder
      ls = [results.pose_landmarks.landmark[landmark_dictionary['LEFT_SHOULDER']].x,
            results.pose_landmarks.landmark[landmark_dictionary['LEFT_SHOULDER']].y,
            results.pose_landmarks.landmark[landmark_dictionary['LEFT_SHOULDER']].z]
      # Left Hip
      lh = [results.pose_landmarks.landmark[landmark_dictionary['LEFT_HIP']].x,
            results.pose_landmarks.landmark[landmark_dictionary['LEFT_HIP']].y,
            results.pose_landmarks.landmark[landmark_dictionary['LEFT_HIP']].z]
      # Left Wirst
      lw = [results.pose_landmarks.landmark[landmark_dictionary['LEFT_WRIST']].x,
            results.pose_landmarks.landmark[landmark_dictionary['LEFT_WRIST']].y,
            results.pose_landmarks.landmark[landmark_dictionary['LEFT_WRIST']].z]
      
      # Average Angle betwen left & right
      #angle = np.mean([get_angle(rs, rh, rw), get_angle(ls, lh, lw)])
      right_angle = get_angle(rs, rh, rw)
      left_angle = get_angle(ls, lh, lw)
      # If Angle greater than 30 degrees
        # Normal Position
      if (right_angle > 30) and (left_angle > 30):
        cv2.putText(image, "Normal", (frame_width  - 150, 30), font, 0.9, green, 2)
      else: 
        cv2.putText(image, "Aero", (frame_width  - 150, 30), font, 0.9, red, 2)
    
    # Write Processed Image to Output Video
    output_video.write(image)

# Release Everything
input_video.release()
output_video.release()
cv2.destroyAllWindows()